# Welcome to the School Achievement Dashboard! 
##### *Produced by Colin Howard and Luke Moffitt for CSCI 77800 -- EthiCS, Fall 2024*

#### For the best user experience, please start by running all of the cells in this notebook (Run > Run All Cells).

# Overview: How to best improve educational outcomes for all students.
* We wanted to establish a link between educational outcomes and relative funding and poverty levels to help inform policy making individuals about the best ways to support education.
* This is important because we are invested in ensuring a quality education for all students, regardless of their income levels.

**Ethically speaking, there is a concern that, as a society, our education system is biased and provides much better service for individuals of means when compared to folks living in poverty.**

* One side claims that all children have an equal right to education, regardless of their means.
* Another side affirms their right to provide the best education possible to them, using whatever means they have at their disposal.

## Possible Solutions
* There are no easy solutions to this problem!
* As the dashboard shows, additional funding does not necessarily produce better results.
* But there may be factors schools and policymakers can attend to to help improve student outcomes.

# Part 1. Poverty and Achievement

In [1]:
%run -i scripts/import.py
%run -i scripts/viz1.py
container1

# Part 2. Poverty and Funding

Although funding has gone up over time in most states, more funding does not always lead to better outcomes.

In [2]:
%run -i scripts/viz2.py
container2

FigureWidget({
    'data': [{'marker': {'color': 'rgb(102, 102, 102)'},
              'name': 'ALABAMA',
              'type': 'scatter',
              'uid': '34a029f4-e65d-46c7-8837-fb8b15a2755d',
              'x': array([2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
                          2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
                          1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987]),
              'y': array([11862, 10728, 10140, 10107, 9717, 9528, 9258, 9146, 9036, 8773, 8577,
                          8726, 8907, 8964, 9197, 8398, 7683, 7073, 6581, 6300, 6029, 5885, 5638,
                          5188, 4849, 4595, 4343, 4109, 3826, 3568, 3415, 3429, 3144, 3019, 2569,
                          2420], dtype=object)},
             {'marker': {'color': 'rgb(204, 80, 62)'},
              'name': 'ALASKA',
              'type': 'scatter',
              'uid': '3e3418b6-1f97-4181-acc2-bbc7

At both the national and state levels, funding per student tends to go down as the percentage of poverty goes up (although there are many outliers in the data). For every 10% increase in childhood poverty, a district spends on average $268 less per student. *Note that this does not take into account local cost of living, which tends to be higher in urban districts with higher rates of poverty.*

In [3]:
%run -i scripts/viz3.py
container3

# Part 3. Factors Affecting Achievement

Here are a select few other variables which also affect achievement. Experiment with the choices to see the results.

In [4]:
%run -i scripts/viz4.py
container4

# Conclusions
With the understanding that higher funding does not neccesarily lead to better outcomes, care must be taken to understand how the money is being spent.

One of the biggest takeaways from this project was the suprise that some schools are spending around 500,000 dollars per year on an individual student. However, even with more than 10x the funding, these schools are not performing 10x as well.

Ultimately, policy makers need to have better information about how schools budgets are broken up. Publicly available data would be more useful if it included additonal data points regarding how the money was spent. As is, it is very difficult to authoritatively discern which factors do and do not directly contribute to student outcomes.


## Summary
1. **Background on your ethical topic, including a debate on the various dimensions/sides of it.** 
   1. **What is a quick overview of this issue?** We wanted to establish a link between educational outcomes and relative funding and poverty levels to help inform policy making individuals about the best ways to support education.
   2. **Why should we be addressing this issue?  (Why is it important?)** This is important because we are invested in ensuring a quality education for all students, regardless of their income levels.
   3. **What are the ethical concerns?** The conern is that, as a society, our education system is biased and provides much better service for individuals of means when compared to folks living in poverty.
   4. **What are the ethical justifications for both sides?** While one side claims that all children have an equal right to education, the other side affirms their right to provide the best education possible to them, using whatever means they have at their disposal.
   5. **What are the possible solutions?** While there are no easy answers to this debate, the truth is that funding does not automatically lead to better results. Instead, the decisions made within school districts about how to spend the money may be more informative. 
2. **References that informed your debate.** We took information from the NAEP (the Nation's Report Card) and from the US Census Bureau to help us make our case.
3. **Introduction of your software solution.** 
   1. **How does your software solution relate to the ethical topic?** This software makes it easy for a user to look at the disparities in funding and educational outcomes in our nation at a variety of levels. It invites thought and discussion about how to best fund our schools, and encourages policy makers to think outside of the box when creating solutions.
   2. **A demonstration of how the software can be used to weigh in on aspects of the ethical topic; perhaps reinforcing the debate in #1.** *See software above!*

4. **Design of your software.** 
   1. **A deeper dive into interesting aspects of the code, the dataset, or the development process that would be educational to others learning about coding in Python, coding with data, or coding for ethics.** Plotly (`plotly`) has a bunch of cool stuff. You can also see how we had to manipulate the data in `pandas` `DataFrame`s to merge the two sets.
   2. **Show us the most complex portions of your code and explain the implementation.** See visualization #3, `function update_poverty()`